In [ ]:
import cv2
import numpy as np

In [ ]:
face_classfier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_extractor(img):
    faces = face_classfier.detectMultiScale(img,1.3,5)
    if faces is ():
        return None
    for(x,y,w,h) in faces:
        cropped_face = img[y:y+h,x:x+w]
        
    return cropped_face
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret,frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame),(200,200))
        file_name_path = 'E://VM RHEL 8 C2//AWORKSPACE//nik//'+ str(count) + '.jpg'
        cv2.imwrite(file_name_path,face)
    else:
        print("Face not found")
        pass
    if cv2.waitKey(1) == 13 or count == 200:
        break
cap.release()
cv2.destroyAllWindows()
print("Sample collection complete !")

In [1]:
from keras.applications import VGG16
rows = 224
cols = 224
model = VGG16(weights = 'imagenet' , include_top=False, input_shape= (rows,cols,3))
model.save('face_recog_vgg.h5')
for layer in model.layers:
    layer.trainable = False
for (i,layer) in enumerate(model.layers):
    print(str(i) + "" + layer.__class__.__name__,layer.trainable)

Using TensorFlow backend.


0InputLayer False
1Conv2D False
2Conv2D False
3MaxPooling2D False
4Conv2D False
5Conv2D False
6MaxPooling2D False
7Conv2D False
8Conv2D False
9Conv2D False
10MaxPooling2D False
11Conv2D False
12Conv2D False
13Conv2D False
14MaxPooling2D False
15Conv2D False
16Conv2D False
17Conv2D False
18MaxPooling2D False


In [2]:
def addlayer(bottom_model , num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)    
    top_model = Dense(512,activation='relu')(top_model)    
    top_model = Dense(num_classes,activation='softmax')(top_model)  
    return top_model

In [3]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [4]:
model.layers

In [5]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
num_classes = 2
FC_Head = addlayer(model,num_classes)
modelnew=Model(inputs=model.input,outputs=FC_Head)
print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [6]:
from keras.preprocessing.image import ImageDataGenerator 

train_data_dir = 'dataset/train/'
validation_data_dir = 'dataset/test/'
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)
train_batchsize = 16
val_batchsize = 10
train_generator =train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=(rows,cols),
                                                  batch_size=train_batchsize,
                                                  class_mode='categorical',
                                                  )
validation_generator =validation_datagen.flow_from_directory(validation_data_dir,
                                                  target_size=(rows,cols),
                                                  batch_size=val_batchsize,
                                                  class_mode='categorical',
                                                  shuffle=False)

Found 475 images belonging to 2 classes.
Found 11 images belonging to 2 classes.


In [7]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint , EarlyStopping
checkpoint=ModelCheckpoint("face_recog_vgg.h5",monitor="val_loss",mode="min", save_best_only = True,verbose=1)
earlystop=EarlyStopping(monitor='val_loss',min_delta = 0, patience=3,verbose=1,restore_best_weights=True)
callbacks=[earlystop,checkpoint]

modelnew.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

nb_train_samples=1190
nb_validation_samples=170
epoches=1
batch_size=16

history = modelnew.fit_generator(train_generator,steps_per_epoch=nb_train_samples // batch_size,
                                epochs=epoches,callbacks=callbacks,
                                validation_data = validation_generator,
                                validation_steps=nb_validation_samples // batch_size)
modelnew.save("face_recog_vgg.h5")

Epoch 1/1
74/74 [==============================] - 488s 7s/step - loss: 0.3145 - accuracy: 0.8999 - val_loss: 3.3614 - val_accuracy: 0.7273

Epoch 00001: val_loss improved from inf to 3.36136, saving model to face_recog_vgg.h5


In [8]:
from keras.models import load_model
classifier = load_model('face_recog_vgg.h5')

In [9]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join



In [10]:
person_dict = {"[0]":"nik","[1]":"yash"}
person_dict_n = {"nik":"nik","yash":"yash"}

In [11]:
def draw_test(name,pred,im):
    person=person_dict[str(pred)]
    BLACK=[0,0,0]
    expanded_image=cv2.copyMakeBorder(im,80,0,0,100,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image,person, (2,60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255),2)
    cv2.imshow(name,expanded_image)

In [14]:
def getRandomImage(path):
    folders=list(filter(lambda x: os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - "+person_dict_n[str(path_class)])
    file_path = path+path_class
    file_names=[f for f in listdir (file_path) if isfile(join (file_path , f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+'/'+image_name)

In [41]:

input_im = getRandomImage("dataset/test/")
input_original=input_im.copy()
input_original = cv2.resize(input_original,None , fx=0.5,fy=0.5,interpolation=cv2.INTER_LINEAR)

input_im = cv2.resize(input_im,(224,224),interpolation=cv2.INTER_LINEAR)
input_im = input_im / 255.
input_im = input_im.reshape(1,224,224,3)

res = np.argmax(classifier.predict(input_im, 1 , verbose = 0), axis=1)

draw_test('Prediction',res,input_original)
cv2.waitKey(5000)
cv2.destroyAllWindows()

Class - yash
